In [1]:
from utils import max_features, maxlen, embed_size_fastText, embed_size_glove, embed_size_glove_twitter
from utils import batch_size, epochs, run_5fold, write_prediction, eval_results
from model import get_textRCNN_model
import pickle
import pandas as pd

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into whic

In [2]:
fastText = pickle.load(open("input/fastText.300d.pickle", "rb"))
glove = pickle.load(open("input/glove.300d.pickle", "rb"))
glove_twitter = pickle.load(open("input/glove.twitter.200d.pickle", "rb"))

In [3]:
x_train = pickle.load(open("input/x_train.pickle", "rb"))
y_train = pickle.load(open("input/y_train.pickle", "rb"))
x_test = pickle.load(open("input/x_test.pickle", "rb"))
submission = pd.read_csv("input/sample_submission.csv")

## check different embedding

In [4]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "num_filter": 50,
    "reg": 0.0,
    "dropout": 0.2,
    "units": 64,
    "kernel_size": 3
}

In [5]:
embeddings = [(fastText, embed_size_fastText, "fastText"), (glove, embed_size_glove, "glove"), (glove_twitter, embed_size_glove_twitter, "glove_twitter")]

In [6]:
predictions = {}

In [7]:
for embed in embeddings:
    print ("\n********************{}********************".format(embed[2]))
    params["embedding_matrix"] = embed[0]
    params["embed_size"] = embed[1]
    y_preds = run_5fold(x_train, y_train, x_test, "loss", get_textRCNN_model, **params)
    predictions[embed[2]] = y_preds


********************fastText********************
Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 606s - loss: 0.0582 - acc: 0.9799 - auc_roc: 0.9242 - val_loss: 0.0442 - val_acc: 0.9831 - val_auc_roc: 0.9755
Epoch 2/50
 - 605s - loss: 0.0403 - acc: 0.9844 - auc_roc: 0.9804 - val_loss: 0.0434 - val_acc: 0.9831 - val_auc_roc: 0.9830
Epoch 3/50
 - 604s - loss: 0.0347 - acc: 0.9863 - auc_roc: 0.9847 - val_loss: 0.0437 - val_acc: 0.9827 - val_auc_roc: 0.9859
Epoch 00003: early stopping
val loss: 0.043651719338119205
val acc: 0.982724922395847
val auc roc: 0.9859407091054837

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 592s - loss: 0.0642 - acc: 0.9771 - auc_roc: 0.8991 - val_loss: 0.0458 - val_acc: 0.9828 - val_auc_roc: 0.9699
Epoch 2/50
 - 591s - loss: 0.0402 - acc: 0.9844 - auc_roc: 0.9766 - val_loss: 0.0426 - val

 - 585s - loss: 0.0578 - acc: 0.9796 - auc_roc: 0.9309 - val_loss: 0.0441 - val_acc: 0.9832 - val_auc_roc: 0.9752
Epoch 2/50
 - 583s - loss: 0.0410 - acc: 0.9841 - auc_roc: 0.9799 - val_loss: 0.0427 - val_acc: 0.9834 - val_auc_roc: 0.9826
Epoch 3/50
 - 583s - loss: 0.0362 - acc: 0.9857 - auc_roc: 0.9843 - val_loss: 0.0417 - val_acc: 0.9839 - val_auc_roc: 0.9855
Epoch 4/50
 - 583s - loss: 0.0324 - acc: 0.9871 - auc_roc: 0.9864 - val_loss: 0.0435 - val_acc: 0.9837 - val_auc_roc: 0.9871
Epoch 00004: early stopping
val loss: 0.04348060028975251
val acc: 0.9837427478973045
val auc roc: 0.9870649405146419

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 596s - loss: 0.0606 - acc: 0.9784 - auc_roc: 0.9168 - val_loss: 0.0440 - val_acc: 0.9832 - val_auc_roc: 0.9727
Epoch 2/50
 - 595s - loss: 0.0413 - acc: 0.9840 - auc_roc: 0.9783 - val_loss: 0.0434 - val_acc: 0.9835 - val_auc_roc: 0.9812
Epoch 3/50
 - 595s - loss: 0.0364 - acc: 0.9856 - auc_roc: 0.9831 - val_loss: 0.0428 - val_

In [8]:
eval_results(predictions)

key: fastText
loss: 0.04325986784804377 (+/- 0.0005836504640900657)
acc: 0.9834002649636574 (+/- 0.00038449136866188275)
roc_auc: 0.9859584322550502 (+/- 0.0011113565519461496)



key: glove
loss: 0.04335984379415399 (+/- 0.001117387021255695)
acc: 0.9833877306351958 (+/- 0.0003284499792237359)
roc_auc: 0.9867472391676323 (+/- 0.0007056296095893186)



key: glove_twitter
loss: 0.04360386898940686 (+/- 0.0010324847360624746)
acc: 0.9834880006617581 (+/- 0.00039773311054037557)
roc_auc: 0.9862272008517914 (+/- 0.001001885184718625)





##### result: 
- fastText: 

loss: 0.04325986784804377 (+/- 0.0005836504640900657)

acc: 0.9834002649636574 (+/- 0.00038449136866188275)

roc_auc: 0.9859584322550502 (+/- 0.0011113565519461496)

- glove: 

loss: 0.04335984379415399 (+/- 0.001117387021255695)

acc: 0.9833877306351958 (+/- 0.0003284499792237359)

roc_auc: 0.9867472391676323 (+/- 0.0007056296095893186)

- glove.twitter: 

loss: 0.04360386898940686 (+/- 0.0010324847360624746)

acc: 0.9834880006617581 (+/- 0.00039773311054037557)

roc_auc: 0.9862272008517914 (+/- 0.001001885184718625)


In [9]:
opt = predictions['fastText']['pred']

In [10]:
write_prediction(opt, submission, "output/textRCNN.csv")

## check num_filter
(suggest values: 100~600: test 100, 300, 500)

In [11]:
filters = [100, 300]  # for the result of 50, refer to the previous log

In [12]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "reg": 0.0,
    "dropout": 0.2,
    "units": 64,
    "kernel_size": 3,
    "embedding_matrix": fastText,
    "embed_size": embed_size_fastText
}

In [ ]:
for f in filters:
    print ("\n********************f = {}********************".format(f))
    params["num_filter"] = f
    y_preds = run_5fold(x_train, y_train, x_test, "loss", get_textRCNN_model, **params)
    predictions[f] = y_preds


********************f = 100********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 595s - loss: 0.0601 - acc: 0.9785 - auc_roc: 0.9193 - val_loss: 0.0436 - val_acc: 0.9831 - val_auc_roc: 0.9739
Epoch 2/50
 - 594s - loss: 0.0397 - acc: 0.9847 - auc_roc: 0.9797 - val_loss: 0.0421 - val_acc: 0.9834 - val_auc_roc: 0.9827
Epoch 3/50
 - 594s - loss: 0.0339 - acc: 0.9868 - auc_roc: 0.9847 - val_loss: 0.0456 - val_acc: 0.9823 - val_auc_roc: 0.9860
Epoch 00003: early stopping
val loss: 0.04555348820217478
val acc: 0.982296706538355
val auc roc: 0.9859510785076997

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 594s - loss: 0.0583 - acc: 0.9793 - auc_roc: 0.9238 - val_loss: 0.0431 - val_acc: 0.9834 - val_auc_roc: 0.9746
Epoch 2/50
 - 594s - loss: 0.0395 - acc: 0.9846 - auc_roc: 0.9799 - val_loss: 0.0427 - val_acc: 0.9838 - val_auc_roc: 0.9827
Epoch 3/50
 - 594s - loss: 0.0337 - acc: 0.9868 - auc_roc: 0.9844 - val_loss: 0.0445 - val_acc: 0.9828 - v

In [ ]:
eval_results(predictions)

###### result: 

- f = 50


- f = 100



- f = 300






In [ ]:
opt

## check different units
(64, 128, 256)

In [ ]:
units = [128, 256]

In [ ]:
for u in units:
    print ("\n********************u:{}********************".format(u))
    params["dense_units"] = u
    predictions[u] = run_5fold_2channels(x_train, y_train, x_test, "loss", get_textRCNN_model, **params)

In [ ]:
eval_results(predictions)

##### result:
- 64: 

- 128:

- 256

In [ ]:
opt = 

#### tuned params

In [ ]:
params_textRCNN = {
    "maxlen": maxlen,
    "max_features": max_features,
    "embedding_size":,
    "embed_matrix":,
    "num_filter": 300,
    "reg": 0.0,
    "dropout": 0.2
}

### write result

In [ ]:
write_prediction(opt, submission, "output/textRCNN.csv")